In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime as dt
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pd_data_base_path = '/content/drive/MyDrive/Preppin Data/Data/'

In [4]:
tx_path = pd.read_csv(pd_data_base_path + 'Transaction Path.csv')
tx_detail = pd.read_csv(pd_data_base_path + 'Transaction Detail.csv')
acc_info = pd.read_csv(pd_data_base_path + 'Account Information.csv')

In [5]:
tx_path.columns = ['Transaction ID', 'Account To', 'Account From']

In [6]:
tx = pd.merge(tx_path, tx_detail, on='Transaction ID', how='left')

In [7]:
tx = tx[tx['Cancelled?']=='N']
tx.drop(columns=['Cancelled?'], inplace=True)

In [8]:
tx_in = tx[['Account To', 'Transaction Date', 'Value']]
tx_out = tx[['Account From', 'Transaction Date', 'Value']]
tx_out['Value'] = tx_out['Value']*-1
tx_in.columns = ['Account Number', 'Balance Date', 'Balance']
tx_out.columns = ['Account Number', 'Balance Date', 'Balance']

In [9]:
acc_info = acc_info[['Account Number', 'Balance Date', 'Balance']]

In [10]:
df = pd.concat([acc_info, tx_in, tx_out], ignore_index=True)

In [11]:
df = df.sort_values(['Account Number', 'Balance Date', 'Balance'], ascending = [True, True, False])
df.columns = ['Account Number', 'Balance Date', 'Transaction Value']

In [12]:
df = df.groupby(['Account Number']).cumsum().merge(df, left_index=True, right_index=True).reset_index(drop=True)

In [13]:
df.columns = ['Balance', 'Account Number', 'Balance Date', 'Transaction Value']
df = df[['Account Number', 'Balance Date', 'Transaction Value', 'Balance']]
df['Transaction Value'] = np.where(df['Balance Date']=='2023-01-31', None, df['Transaction Value'])
df.head()

,Account Number,Balance Date,Transaction Value,Balance
0,10005367,2023-01-31,None,728.25
1,10005367,2023-02-02,-1097.6,-369.35
2,10005367,2023-02-04,144.75,-224.60
3,10005367,2023-02-04,-67.01,-291.61
4,10005367,2023-02-05,-108.26,-399.87
